(C) 2024 Cadence Design Systems, Inc. (Cadence)
All rights reserved.

TERMS FOR USE OF SAMPLE CODE The software below ("Sample Code") is
provided to current licensees or subscribers of Cadence products or
SaaS offerings (each a "Customer").
Customer is hereby permitted to use, copy, and modify the Sample Code,
subject to these terms. Cadence claims no rights to Customer's
modifications. Modification of Sample Code is at Customer's sole and
exclusive risk. Sample Code may require Customer to have a then
current license or subscription to the applicable Cadence offering.
THE SAMPLE CODE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
EXPRESS OR IMPLIED.  OPENEYE DISCLAIMS ALL WARRANTIES, INCLUDING, BUT
NOT LIMITED TO, WARRANTIES OF MERCHANTABILITY, FITNESS FOR A
PARTICULAR PURPOSE AND NONINFRINGEMENT. In no event shall Cadence be
liable for any damages or liability in connection with the Sample Code
or its use.

# A tour of `westpa.analysis`

The `analysis` subpackage provides a lightweight Python API for interacting with west.h5 files and accessing saved trajectory data. It is designed with interactive computing in mind, where the goal might be exploratory analysis or prototyping a new analysis method. Note that, unlike the CLI tools, `westpa.analyis` is unaware of west.cfg files and the analysis schemes defined therein.

## Data organization

The API is organized around the `Run` data type, which represents a WE simulation run. A `Run` object can be created by passing a west.h5 file path to the `open()` class method:

In [ ]:
from westpa.analysis import Run

run = Run.open("../ntl9_sample_files/completed_files/west.h5")
run

A `Run` object is an iterable of `Iteration` objects (one for each *completed* WE iteration). Each `Iteration` object is an iterable of `Walker` objects:

In [ ]:
for iteration in run:
    for walker in iteration:
        ...
walker

Iterations and walkers can also be accessed individually, e.g.,  

In [ ]:
run.iteration(10).walker(4)

## Summary tables

The iteration summary table for a run can be accessed using the `summary` attribute:

In [ ]:
run.summary[:10]

The segment summary table ("seg_index") for an iteration can be accessed using the `segment_summaries` attribute:

In [ ]:
run.iteration(1).segment_summaries

Iterations also have `basis_state_summaries` and `target_state_summaries` attributes.

## Weights and progress coordinates

Weights and progress coordinates can be accessed on a per-iteration basis:

In [ ]:
iteration = run.iteration(1)
iteration.weights, iteration.pcoords

or on a per-walker basis:

In [ ]:
walker = run.iteration(1).walker(0)
walker.weight, walker.pcoords

**Performance consideration:** It is generally much faster to load the progress coordinates for an iteration all at once, rather than by looping over individual walkers:

In [ ]:
%%timeit
for iteration in run:
    _ = iteration.pcoords

In [ ]:
%%timeit
for iteration in run:
    for walker in iteration:
        _ = walker.pcoords

## Bins

The bin mapper used for resampling (regarded as the first step of an iteration) is given by the `bin_mapper` attribute:

In [ ]:
run.iteration(2).bin_mapper

The bin mapper defines a set of bins:

In [ ]:
bins = list(run.iteration(2).bins)
bins

Here, a `Bin` object is a [container](https://docs.python.org/3/library/collections.abc.html#collections.abc.Container) representing a subset of progress coordinate space. For non-adaptive bin mappers that always map a given progress coordinate point to the same bin index, bin membership can be checked using the `in` operator:

In [ ]:
walker = run.iteration(1).walker(0)
walker.pcoords[0] in bins[-1]  # True if the walker started in the last bin

Given a bin mapper and a set of target states, the *sink* is the union of bins that contain at least one target state. 

In [ ]:
run.iteration(1).sink

Like a `Bin` object, a `BinUnion` object is a container representing the subset of progress coordinate space. For example, the following loop verifies that the final progress coordinates of recycled walkers fall inside the sink, while the final progress coordinates of non-recycled walkers fall outside the sink:

In [ ]:
for iteration in run:
    for walker in iteration:
        if walker.recycled:
            assert walker.pcoords[-1] in iteration.sink
        else:
            assert walker.pcoords[-1] not in iteration.sink

## Tracing history

Each walker has `parent` and `children` attributes. The `parent` can be either a walker from the previous iteration:

In [ ]:
run.iteration(2).walker(0).parent

or an initial state:

In [ ]:
run.iteration(1).walker(0).parent

The `children` attribute is an iterable of walkers:

In [ ]:
list(run.iteration(1).walker(0).children)

The lineage of a walker can be traced back along parents using the `trace()` method:

In [ ]:
walker = max(run.recycled_walkers, key=lambda x: x.weight)
trace = walker.trace()
trace

The returned `Trace` object is an iteratable of walkers:

In [ ]:
list(trace)

By default, the `trace()` method traces back to an initial state, given by the `initial_state` attribute:

In [ ]:
trace.initial_state

It is also possible to trace no further than `max_length` iterations:

In [ ]:
trace = walker.trace(max_length=5)
list(trace)

or back to a given `source` region in progress coordinate space:

In [ ]:
class UpperHalfLine:
    def __init__(self, x0):
        self.x0 = x0
        
    def __contains__(self, x):
        return x > self.x0

trace = walker.trace(source=UpperHalfLine(5.0))
list(trace)

The *history graph* is the directed graph with an edge pointing from each walker to its parent:

In [ ]:
import networkx as nx

def history_graph(run):
    return nx.DiGraph((walker, walker.parent) for walker in run.walkers)

By taking powers of the adjacency matrix of this graph, we can obtain information about paths (traces) of a given length. This can be useful when constructing a Markov state model with a lag time that is an integer multiple of the resampling time.

In [ ]:
from scipy import sparse

def lagged_pairs(run, lag=1):
    graph = history_graph(run)
    nodes = list(graph.nodes)
    matrix = sparse.linalg.matrix_power(nx.adjacency_matrix(graph), lag)
    for i, j, _ in zip(*sparse.find(matrix)):
        yield nodes[j], nodes[i]

pairs = list(lagged_pairs(run, lag=3))
pairs[:10]

## Recycling

The `recycled_walkers` attribute provides an iterator over all recycled walkers in a run or iteration. For example,

In [ ]:
max(run.recycled_walkers, key=lambda x: x.weight)

gives the highest weighted recycled walker, and

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot as plt

plt.plot(
    [iteration.number for iteration in run],
    [sum(walker.weight for walker in iteration.recycled_walkers) for iteration in run],
)
plt.xlabel("WE Iteration")
plt.ylabel("Recycled Weight")
plt.show()

plots the total weight recycled at each WE iteration.

## Loading trajectories

The `HDF5MDTrajectory` reader can be used to access trajectory data saved using the HDF5 framework:

In [ ]:
from westpa.analysis import HDF5MDTrajectory

trajectory = HDF5MDTrajectory()

Here, `trajectory` is a callable that takes a walker or trace as input and returns the corresponding trajectory as an `mdtraj.Trajectory` object. E.g.,

In [ ]:
walker = run.iterations[-1].walker(0)
trajectory(walker)

In [ ]:
traj = trajectory(walker.trace())
traj

In [ ]:
import nglview

view = nglview.show_mdtraj(traj)
view

## Closing a run

The `close()` method closes a run and its underlying west.h5 file:

In [ ]:
run.close()
run

In [ ]:
run.h5file

A run can also be used as a context manager. In this case, it will be closed automatically when execution leaves the context:

In [ ]:
with Run.open("../ntl9_sample_files/completed_files/west.h5") as run:
    pass

run.h5file